In [1]:
import pandas as pd 
import re
import email

In [2]:
#Loading File
df = pd.read_csv("Data/emails.csv")

In [3]:
# create list of email objects
emails = list(map(email.parser.Parser().parsestr,df['message']))

# extract headings such as subject, from, to etc..
headings  = emails[0].keys()

# Goes through each email and grabs info for each key
# doc['From'] grabs who sent email in all emails
for key in headings:
    df[key] = [doc[key] for doc in emails]

In [4]:
df.head()

,file,message,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf


In [5]:
# Function to get the raw text from the email
def get_raw_text(emails):
    email_text = []
    for email in emails.walk():
        if email.get_content_type() == 'text/plain':
            email_text.append(email.get_payload())
    return ''.join(email_text)

df['body'] = list(map(get_raw_text, emails))
df.head()
df['user'] = df['file'].map(lambda x: x.split('/')[0])

In [6]:
def clean_column(data):
    if data is not None:

        # Remove new line characters and mail headers
        data = re.sub('re:', '', data)
        data = re.sub('-', '', data)
        data = re.sub('_', '', data)
        # Remove data between square brackets and new line characters
        data =re.sub('\[[^]]*\]', '', data)
        data = re.sub(r'\n',' ',data)
        # strip html 
        p = re.compile(r'<.*?>')
        data = re.sub(r"\'ve", " have ", data)
        data = re.sub(r"can't", "cannot ", data)
        data = re.sub(r"n't", " not ", data)
        data = re.sub(r"I'm", "I am", data)
        data = re.sub(r" m ", " am ", data)
        data = re.sub(r"\'re", " are ", data)
        data = re.sub(r"\'d", " would ", data)
        data = re.sub(r"\'ll", " will ", data)
        data = re.sub('forwarded by phillip k allenhouect on    pm', '',data)
        data = re.sub(r"httpitcappscorpenroncomsrrsauthemaillinkaspidpage", "", data)
        
        data = p.sub('', data)
        if 'forwarded by:' in data:
            data = data.split('subject')[1]
        data = data.strip()
        return data
    return 'No Subject'


df['Subject_new'] = df['Subject'].apply(clean_column)
df['body_new'] = df['body'].apply(clean_column)

<>:9: SyntaxWarning: invalid escape sequence '\['
<>:9: SyntaxWarning: invalid escape sequence '\['
/var/folders/tv/p9y5281j6zq5phnf4p1l47dw0000gn/T/ipykernel_49775/4253332173.py:9: SyntaxWarning: invalid escape sequence '\['
  data =re.sub('\[[^]]*\]', '', data)


In [7]:
# Selecting rows to drop after looking at the data
drop_list = []
for i in df['body_new'].index:
    if "\t" in df['body_new'][i]:
        drop_list.append(i)
    if ("Forwarded by" in df['body_new'][i]) or ("forwarded by" in df['body_new'][i]):
        drop_list.append(i)
    if "Outlook Migration Team@ENRON" in df['body_new'][i]:
        drop_list.append(i)
    if ("=09" in df['body_new'][i]) or ("=20" in df['body_new'][i]):
        drop_list.append(i)

In [8]:
# Dropping a lot of rows since we do not need a large dataset
len(drop_list)

257573

In [9]:
# Droping the rows
df.drop(df.index[drop_list], inplace=True)

In [ ]:
# Manually looking at the data to see if more cleaning is needed
for x in df['body_new'][:1000]:

    print(x)
    print("===============")

In [11]:
#Show the top 20 value counts
df["From"].value_counts().head(20)

From
pete.davis@enron.com             9143
kay.mann@enron.com               8331
vince.kaminski@enron.com         5648
jeff.dasovich@enron.com          5519
enron.announcements@enron.com    5446
chris.germany@enron.com          5197
matthew.lenhart@enron.com        4119
kate.symes@enron.com             4117
tana.jones@enron.com             3652
debra.perlingiere@enron.com      3633
sara.shackleton@enron.com        3543
eric.bass@enron.com              3184
benjamin.rogers@enron.com        2858
no.address@enron.com             2659
susan.scott@enron.com            2649
john.arnold@enron.com            2296
gerald.nemec@enron.com           2229
robin.rodrigue@enron.com         2138
sally.beck@enron.com             2024
phillip.love@enron.com           2018
Name: count, dtype: int64

In [12]:
# Create dataframe with mails from the top 18 senders of the company to avoid advertising and automated messages
top_adresses = list(df["From"].value_counts().head(20).index)
top_adresses.remove('enron.announcements@enron.com')
top_adresses.remove('no.address@enron.com')
df_top = df[df["From"].isin(top_adresses)]

In [ ]:
# Manually looking at the data to see if more cleaning is needed
for x in df_top['body_new'][:1000]:

    print(x)
    print("===============")

In [14]:
#export as pickle file
df_top.to_pickle("Data/df_selection.pkl")